<a href="https://colab.research.google.com/github/TatyanaLanceva/Netology/blob/main/6_4_Big_Data_Analytics_Python_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание Python


1.   Загрузите в колаб файлы по [оценкам](https://drive.google.com/file/d/19Ey1uDH8yMk2jiJE3OQ098MeOStv8OPr/view?usp=sharing) (ratings) и [фильмам](https://drive.google.com/file/d/1aRF9U3ZVpmCCiT7RYAV_8UoOengHzsHL/view?usp=sharing) (movies) и создайте на их основе pandas-датафреймы

2.   Средствами Pandas, используя dataframe ratings, найдите id пользователя, поставившего больше всего оценок


3.   Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь


4.   Добавьте к датафрейму из задания 3 столбцы:
	По жанрам. Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет


5.   Сформируйте X_train, X_test, y_train, y_test

6.   Возьмите модель линейной регрессии (или любую другую для задачи регрессии)  и обучите ее на фильмах

7.   Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии

8.   Загрузить данные в spark


9.   Средствами спарка вывести среднюю оценку для каждого фильма


10.   Посчитайте средствами спарка среднюю оценку для каждого жанра


11.   В спарке получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок, либо по самой оценке - на Ваш выбор)














In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor


In [ ]:
# Добавим название столбцов согласно Readme в файл с фильмами
column_names = [
    'movie id', 'movie title', 'release date', 'video release date',
    'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation',
    "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
    'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
    'Thriller', 'War', 'Western'
]

# Чтение файла с добавлением столбцов
movies = pd.read_csv(
    '/content/u.item.csv',
    encoding='ISO-8859-1',
    sep='|',
    header=None,
    names=column_names,
    on_bad_lines='skip'
)
# Удаление столбца 'video release date', он пустой
movies = movies.drop(columns=['video release date'])

# Выведем таблицу фильмов
movies.head()

,movie id,movie title,release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
# Добавим названия столбцов к таблице с рейтингами согласно Readme
column_names = ['user id', 'item id', 'rating', 'timestamp']

# Чтение файла с рейтингами
raiting = pd.read_csv(
    '/content/u.data.csv',
    encoding='ISO-8859-1',
    sep='\t',  # Разделитель — табуляция
    header=None,
    names=column_names,
    on_bad_lines='skip'
)
# Вывод
print(raiting.head())

   user id  item id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596


In [ ]:
# Объединим таблицы в общую для дальнейшей работы
merged_data = pd.merge(
    movies,
    raiting,
    left_on='movie id',
    right_on='item id',
    how='inner'
)

# Удалим дублирующий столбец 'item id'
merged_data = merged_data.drop(columns=['item id'])

# Вывод
merged_data.head()

,movie id,movie title,release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,user id,rating,timestamp
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,308,4,887736532
1,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,287,5,875334088
2,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,148,4,877019411
3,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,280,4,891700426
4,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,66,3,883601324


In [ ]:
# Проверим пропуски, чтобы понимать, нужна ли обработка
missing_values = merged_data.isnull().sum()
# Вывод результата
print(missing_values)
# Пропуски есть только в столбце IMDb URL, если он потребуется для анализа, можно будет обработать

movie id         0
movie title      0
release date     9
IMDb URL        13
unknown          0
Action           0
Adventure        0
Animation        0
Children's       0
Comedy           0
Crime            0
Documentary      0
Drama            0
Fantasy          0
Film-Noir        0
Horror           0
Musical          0
Mystery          0
Romance          0
Sci-Fi           0
Thriller         0
War              0
Western          0
user id          0
rating           0
timestamp        0
dtype: int64


In [ ]:
# Группировка данных по 'user id' и подсчет количества оценок для каждого пользователя
user_rating_counts = merged_data.groupby('user id').size()

# Нахождение пользователя с максимальным количеством оценок
max_user_id = user_rating_counts.idxmax()
max_ratings_count = user_rating_counts[max_user_id]

# Вывод результата
print(f"Пользователь с ID {max_user_id} поставил больше всего оценок: {max_ratings_count}")

# Такой же результат был получен при выполнении задания в  GoogleSheets

Пользователь с ID 405 поставил больше всего оценок: 737


In [ ]:
# Указываем ID пользователя, которого мы хотим проанализировать, это 405 номер
target_user_id = 405

# Фильтрация: оставляем только строки, где 'user id' равен target_user_id
filtered_ratings = merged_data[merged_data['user id'] == target_user_id]

# Вывод результатов
print(f"Фильмы, оцененные пользователем с ID {target_user_id}:\n")
print(filtered_ratings.to_string(index=False))  # index=False убирает индексы
print("\n")  # Добавляем пустую строку для разделения

# Проверка количества оценок
num_ratings = filtered_ratings.shape[0]
print(f"Пользователь с ID {target_user_id} оценил {num_ratings} фильмов.\n")

# Получение списка ID фильмов
rated_movies = filtered_ratings['movie id'].tolist()

# Вывод списка ID фильмов с переносами строк
print(f"ID фильмов, оцененных пользователем с ID {target_user_id}:")
print(rated_movies, sep=", ")

Фильмы, оцененные пользователем с ID 405:

 movie id                                                         movie title release date                                                                                                                       IMDb URL  unknown  Action  Adventure  Animation  Children's  Comedy  Crime  Documentary  Drama  Fantasy  Film-Noir  Horror  Musical  Mystery  Romance  Sci-Fi  Thriller  War  Western  user id  rating  timestamp
        2                                                    GoldenEye (1995)  01-Jan-1995                                                                            http://us.imdb.com/M/title-exact?GoldenEye%20(1995)        0       1          1          0           0       0      0            0      0        0          0       0        0        0        0       0         1    0        0      405       1  885547953
        4                                                   Get Shorty (1995)  01-Jan-1995                             

In [ ]:
# Вычислим статистику по фильмам
movie_stats = merged_data.groupby('movie id').agg(
    total_ratings=('rating', 'count'),  # Общее количество оценок
    sum_ratings=('rating', 'sum')       # Суммарная оценка
).reset_index()

# Добавим столбец с средним рейтингом
movie_stats['average_rating'] = movie_stats['sum_ratings'] / movie_stats['total_ratings']

# Вывод первых строк статистики
print(movie_stats.head())

   movie id  total_ratings  sum_ratings  average_rating
0         1            452         1753        3.878319
1         2            131          420        3.206107
2         3             90          273        3.033333
3         4            209          742        3.550239
4         5             86          284        3.302326


In [ ]:
# Добавляем данные по рейтингу и фильтруем по пользователю, столбцы по жанрам были у нас изначально из-за объединения таблиц
rated_movies = pd.merge(
    filtered_ratings,  # Фильтрованные данные по пользователю
    movie_stats[['movie id', 'average_rating']],  # Только нужные столбцы из статистики
    on='movie id',     # Ключ для объединения
    how='left'         # Тип объединения (left join)
)
rated_movies = rated_movies[rated_movies['user id'] == 405]
rated_movies.head()


,movie id,movie title,release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Mystery,Romance,Sci-Fi,Thriller,War,Western,user id,rating,timestamp,average_rating
0,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,1,0,0,405,1,885547953,3.206107
1,4,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,405,4,885547314,3.550239
2,5,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,1,0,0,405,4,885545070,3.302326
3,8,Babe (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Babe%20(1995),0,0,0,0,1,1,...,0,0,0,0,0,0,405,4,885545015,3.995434
4,11,Seven (Se7en) (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Se7en%20(1995),0,0,0,0,0,0,...,0,0,0,1,0,0,405,4,885545263,3.847458


In [ ]:
rated_movies.info() # Проверим количетство строк, мы видим, что пользователь оценил 737 фильмов, что соответствует данным уже полученным в GoogleSheets

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 737 entries, 0 to 736
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   movie id        737 non-null    int64  
 1   movie title     737 non-null    object 
 2   release date    737 non-null    object 
 3   IMDb URL        736 non-null    object 
 4   unknown         737 non-null    int64  
 5   Action          737 non-null    int64  
 6   Adventure       737 non-null    int64  
 7   Animation       737 non-null    int64  
 8   Children's      737 non-null    int64  
 9   Comedy          737 non-null    int64  
 10  Crime           737 non-null    int64  
 11  Documentary     737 non-null    int64  
 12  Drama           737 non-null    int64  
 13  Fantasy         737 non-null    int64  
 14  Film-Noir       737 non-null    int64  
 15  Horror          737 non-null    int64  
 16  Musical         737 non-null    int64  
 17  Mystery         737 non-null    int

In [ ]:
# Преобразуем timestamp в год
rated_movies['year'] = pd.to_datetime(rated_movies['timestamp'], unit='s').dt.year
# Удалим ненужные столбцов
rated_movies = rated_movies.drop(columns=['movie title', 'IMDb URL', 'release date'], errors='ignore')

# Целевая переменная (оценка фильма пользователем)
y = rated_movies['rating']

# Признаки: год выхода, жанры, общее количество оценок, суммарная оценка
X = rated_movies.drop(columns=['rating', 'user id', 'timestamp', 'average_rating'])

# Проверка типов данных
print(X.dtypes)


movie id       int64
unknown        int64
Action         int64
Adventure      int64
Animation      int64
Children's     int64
Comedy         int64
Crime          int64
Documentary    int64
Drama          int64
Fantasy        int64
Film-Noir      int64
Horror         int64
Musical        int64
Mystery        int64
Romance        int64
Sci-Fi         int64
Thriller       int64
War            int64
Western        int64
year           int32
dtype: object


In [ ]:
# Разделеним данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Обученим модели линейной регрессии
model = LinearRegression()
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка качества модели
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

# Вывод результатов
print("Средняя абсолютная ошибка (MAE):", mae)
print("Среднеквадратичная ошибка (MSE):", mse)
print("Корень из среднеквадратичной ошибки (RMSE):", rmse)

Средняя абсолютная ошибка (MAE): 1.0437638515599126
Среднеквадратичная ошибка (MSE): 1.7687112094637603
Корень из среднеквадратичной ошибки (RMSE): 1.3299290242203756


In [ ]:
# Целевая переменная (оценка фильма пользователем)
y = rated_movies['rating']

# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Обучение модели случайного леса
# Инициализация модели случайного леса
model = RandomForestRegressor(
    n_estimators=100,  # Количество деревьев в лесу
    random_state=42,   # Фиксация случайного seed для воспроизводимости
    n_jobs=-1          # Использование всех доступных ядер CPU
)

# Обучение модели на обучающей выборке
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка качества модели
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5  # Корень из MSE

# Вывод результатов
print("Средняя абсолютная ошибка (MAE):", mae)
print("Среднеквадратичная ошибка (MSE):", mse)
print("Корень из среднеквадратичной ошибки (RMSE):", rmse)

Средняя абсолютная ошибка (MAE): 0.9808783783783784
Среднеквадратичная ошибка (MSE): 1.8821885135135135
Корень из среднеквадратичной ошибки (RMSE): 1.3719287567193543


Вывод: обе модели показывают умеренное качество. Случайный лес немного лучше по MAE, а линейная регрессия немного лучше по MSE и RMSE. Для улучшения качества стоит добавить признаки или рассмотреть другие модели.

# Вторая часть практики Python связана со Spark’ом

1.   Загрузить данные в spark

2.   Средствами спарка вывести среднюю оценку для каждого фильма

3.   Посчитайте средствами спарка среднюю оценку для каждого жанра

4.   В спарке получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок, либо по самой оценке - на Ваш выбор)






# Установка Spark

In [ ]:
!apt-get update

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,767 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,679 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://downloads.apache.org/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz

In [ ]:
!tar -xvf spark-3.5.5-bin-hadoop3.tgz

spark-3.5.5-bin-hadoop3/
spark-3.5.5-bin-hadoop3/jars/
spark-3.5.5-bin-hadoop3/jars/HikariCP-2.5.1.jar
spark-3.5.5-bin-hadoop3/jars/JLargeArrays-1.5.jar
spark-3.5.5-bin-hadoop3/jars/JTransforms-3.1.jar
spark-3.5.5-bin-hadoop3/jars/RoaringBitmap-0.9.45.jar
spark-3.5.5-bin-hadoop3/jars/ST4-4.0.4.jar
spark-3.5.5-bin-hadoop3/jars/activation-1.1.1.jar
spark-3.5.5-bin-hadoop3/jars/aircompressor-0.27.jar
spark-3.5.5-bin-hadoop3/jars/algebra_2.12-2.0.1.jar
spark-3.5.5-bin-hadoop3/jars/annotations-17.0.0.jar
spark-3.5.5-bin-hadoop3/jars/antlr-runtime-3.5.2.jar
spark-3.5.5-bin-hadoop3/jars/antlr4-runtime-4.9.3.jar
spark-3.5.5-bin-hadoop3/jars/aopalliance-repackaged-2.6.1.jar
spark-3.5.5-bin-hadoop3/jars/arpack-3.0.3.jar
spark-3.5.5-bin-hadoop3/jars/arpack_combined_all-0.1.jar
spark-3.5.5-bin-hadoop3/jars/arrow-format-12.0.1.jar
spark-3.5.5-bin-hadoop3/jars/arrow-memory-core-12.0.1.jar
spark-3.5.5-bin-hadoop3/jars/arrow-memory-netty-12.0.1.jar
spark-3.5.5-bin-hadoop3/jars/arrow-vector-12.0.1.jar


In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.5-bin-hadoop3"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

# Работа с данными

In [ ]:
# Загружаем данные с указанием разделителя '|'
movies = spark.read.csv('/content/u.item.csv', inferSchema=True, header=False, sep='|')

# Переименовываем столбцы
column_names = [
    "movie id", "movie title", "release date", "video release date",
    "IMDb URL", "unknown", "Action", "Adventure", "Animation",
    "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
    "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi",
    "Thriller", "War", "Western"
]

movies = movies.toDF(*column_names)

# Просмотр первых 5 строк
movies.show(5)

+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie id|      movie title|release date|video release date|            IMDb URL|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1| Toy Story (1995)| 01-Jan-1995|              NULL|http://us.imdb.co...|      0|     0|        0|        1|         1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|       2| GoldenEye (1995)| 01-Jan-1995|              N

In [ ]:
# Загружаем данные с указанием разделителя '\t'
raiting = spark.read.csv('/content/u.data.csv', inferSchema=True, header=False, sep='\t')

# Переименовываем столбцы
column_names = ["user id", "item id", "rating", "timestamp"]

raiting = raiting.toDF(*column_names)

# Просмотр первых 5 строк
raiting.show(5)

+-------+-------+------+---------+
|user id|item id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
+-------+-------+------+---------+
only showing top 5 rows



In [ ]:
merged_data = movies.join(raiting, movies["movie id"] == raiting["item id"], how="left")
merged_data.show(5)

+--------+----------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+-------+-------+------+---------+
|movie id|     movie title|release date|video release date|            IMDb URL|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|user id|item id|rating|timestamp|
+--------+----------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+-------+-------+------+---------+
|       1|Toy Story (1995)| 01-Jan-1995|              NULL|http://us.imdb.co...|      0|     0|        0|        1|         1|     1|    0|          0|    0|      0|        0|     0|      0|      

In [ ]:
# Выполним группировку по ID и названию фильма, вычислим среднюю оценку
from pyspark.sql import functions as F

average_ratings = merged_data.groupBy("movie id", "movie title") \
    .agg(F.round(F.avg("rating"), 2).alias("average_rating")) \
    .orderBy("movie id")  # Cортировка по ID

# Вывод результата
average_ratings.show()

+--------+--------------------+--------------+
|movie id|         movie title|average_rating|
+--------+--------------------+--------------+
|       1|    Toy Story (1995)|          3.88|
|       2|    GoldenEye (1995)|          3.21|
|       3|   Four Rooms (1995)|          3.03|
|       4|   Get Shorty (1995)|          3.55|
|       5|      Copycat (1995)|           3.3|
|       6|Shanghai Triad (Y...|          3.58|
|       7|Twelve Monkeys (1...|           3.8|
|       8|         Babe (1995)|           4.0|
|       9|Dead Man Walking ...|           3.9|
|      10|  Richard III (1995)|          3.83|
|      11|Seven (Se7en) (1995)|          3.85|
|      12|Usual Suspects, T...|          4.39|
|      13|Mighty Aphrodite ...|          3.42|
|      14|  Postino, Il (1994)|          3.97|
|      15|Mr. Holland's Opu...|          3.78|
|      16|French Twist (Gaz...|          3.21|
|      17|From Dusk Till Da...|          3.12|
|      18|White Balloon, Th...|           2.8|
|      19|Ant

In [ ]:
# Посчитайте средствами спарка среднюю оценку для каждого жанра
genres = [
    "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]

# Вычисляем среднюю оценку для каждого жанра
genre_avg_exprs = [
    F.round(F.avg(F.when(F.col(genre) == 1, F.col("rating"))), 2).alias(genre)
    for genre in genres
]

# Агрегируем данные
genre_avg = merged_data.agg(*genre_avg_exprs).first()

# Преобразуем результат в формат (жанр, средняя_оценка)
genre_avg_list = [(genre, genre_avg[genre]) for genre in genres]

# Создаем итоговый датафрейм
result_df = spark.createDataFrame(genre_avg_list, ["genre", "average_rating"]).orderBy("genre")

# Выводим результат
result_df.show(truncate=False)

+-----------+--------------+
|genre      |average_rating|
+-----------+--------------+
|Action     |3.48          |
|Adventure  |3.5           |
|Animation  |3.58          |
|Children's |3.35          |
|Comedy     |3.39          |
|Crime      |3.63          |
|Documentary|3.67          |
|Drama      |3.69          |
|Fantasy    |3.22          |
|Film-Noir  |3.92          |
|Horror     |3.29          |
|Musical    |3.52          |
|Mystery    |3.64          |
|Romance    |3.62          |
|Sci-Fi     |3.56          |
|Thriller   |3.51          |
|War        |3.82          |
|Western    |3.61          |
|unknown    |3.2           |
+-----------+--------------+



In [ ]:
# Вычисляем среднюю оценку для каждого фильма
movie_avg_ratings = merged_data.groupBy("movie id", "movie title").agg(
    F.round(F.avg("rating"), 2).alias("average_rating")
)

# Получаем топ-5 лучших фильмов (по средней оценке)
top_5_best_movies = movie_avg_ratings.orderBy("average_rating", ascending=False).limit(5)

# Получаем топ-5 худших фильмов (по средней оценке)
top_5_worst_movies = movie_avg_ratings.orderBy("average_rating", ascending=True).limit(5)

# Выводим результаты
print("Топ-5 лучших фильмов по оценке:")
top_5_best_movies.show(truncate=False)

print("Топ-5 худших фильмов по оценке:")
top_5_worst_movies.show(truncate=False)

Топ-5 лучших фильмов по оценке:
+--------+------------------------------------------+--------------+
|movie id|movie title                               |average_rating|
+--------+------------------------------------------+--------------+
|1467    |Saint of Fort Washington, The (1993)      |5.0           |
|1201    |Marlene Dietrich: Shadow and Light (1996) |5.0           |
|1599    |Someone Else's America (1995)             |5.0           |
|814     |Great Day in Harlem, A (1994)             |5.0           |
|1189    |Prefontaine (1997)                        |5.0           |
+--------+------------------------------------------+--------------+

Топ-5 худших фильмов по оценке:
+--------+--------------------------------------------------+--------------+
|movie id|movie title                                       |average_rating|
+--------+--------------------------------------------------+--------------+
|314     |3 Ninjas: High Noon At Mega Mountain (1998)       |1.0           |
|1577 